In [286]:
import pandas as pd
import numpy as np
import os
import glob

### Creamos el datamart final para entrenar el modelo

In [287]:
def load_homes_external(datalake_path):
    """
    Carga y une los archivos homes.tsv y external_indicators.tsv de todos los años en un único DataFrame.
    La unión se realiza por 'Año' y 'CCAA'.
    """
    all_homes = glob.glob(os.path.join(datalake_path, "*", "homes.tsv"))
    all_external = glob.glob(os.path.join(datalake_path, "*", "external_indicators.tsv"))

    df_list = []

    for home_file, external_file in zip(all_homes, all_external):
        year = os.path.basename(os.path.dirname(home_file))  # Extraer el año de la ruta

        df_homes = pd.read_csv(home_file, sep="\t")
        df_external = pd.read_csv(external_file, sep="\t")

        df_homes["Año"] = int(year)
        df_external["Año"] = int(year)

        # Unir por Año y CCAA
        df_merged = pd.merge(df_homes, df_external, on=["Año", "CCAA"], how="inner")
        df_list.append(df_merged)

    if not df_list:
        raise ValueError("No se encontraron archivos homes.tsv o external_indicators.tsv en la ruta del datalake.")

    # Concatenar todos los DataFrames
    df_homes_external = pd.concat(df_list, ignore_index=True)
    return df_homes_external

In [288]:
def merge_with_datamart(df_homes_external, datamart_path):
    """
    Une el DataFrame combinado de homes y external_indicators con el datamart de supercategorías.
    La unión se realiza por 'Año' y 'NUMERO'. Cada supercategoría se convierte en una columna
    con el gasto mensual correspondiente.
    """
    df_datamart = pd.read_csv(datamart_path, sep="\t")

    # Pivotar el datamart: cada SUPER_CATEGORIA como columna
    df_pivot = df_datamart.pivot_table(
        index=["Año", "NUMERO"],
        columns="SUPER_CATEGORIA",
        values="GASTOMON",
    ).reset_index()

    # Renombrar columnas para mayor claridad, por ejemplo: SC_011, SC_012, etc.
    df_pivot.columns = ['Año', 'NUMERO'] + [f"SC_{col}" for col in df_pivot.columns[2:]]

    # Unir con el DataFrame combinado
    df_final = pd.merge(df_homes_external, df_pivot, on=["Año", "NUMERO"], how="inner")
    return df_final


In [289]:
load_homes_external_path = "../../DataLake" 
datamart_path = "../../datamarts/datamart_supercategories_three_digits.tsv"
output_path = "../../datamarts/final_dataframe.tsv" 

# Cargar y unir los archivos homes y external_indicators
df_homes_external = load_homes_external(load_homes_external_path)

# Unir con el datamart
df_final = merge_with_datamart(df_homes_external, datamart_path)


In [290]:
pd.set_option('display.max_columns', None)  
df_final.head(15)  # Mostrar las primeras filas del DataFrame final

,NUMERO,CCAA,CAPROV,TAMAMU,DENSIDAD,NUMACTI,NUMOCU,NNINOSD,NHIJOSD,NUMESTU,NUMNOESTU,EDADSP,REGTEN,ZONARES,SUPERF,AGUACALI,CALEF,TIPOCASA,DISPOSIOV,NUMOVD,IMPEXAC,FUENPRINRED,GASTOT,GASTMON,GASTNOMON,FACTOR,NUMPERI,COMITOT,NADUL_MAS,NADUL_FEM,NNINO_MAS,NNINO_FEM,ANC_MAS,ANC_FEM,PERCEP_MAS,NO_PERCEP_MAS,PERCEP_FEM,NO_PERCEP_FEM,NACION_ESP,NACION_EXT,EDUC_SUPERIOR,Año,Tasa_Paro,T_med,Inflacion,Tipo_Interes,EUR_USD,IPC,SC_11,SC_12,SC_21,SC_22,SC_31,SC_32,SC_41,SC_42,SC_43,SC_44,SC_45,SC_51,SC_52,SC_53,SC_54,SC_55,SC_56,SC_61,SC_62,SC_63,SC_71,SC_72,SC_73,SC_81,SC_82,SC_83,SC_91,SC_92,SC_93,SC_94,SC_95,SC_96,SC_101,SC_102,SC_103,SC_104,SC_111,SC_112,SC_121,SC_123,SC_124,SC_125,SC_126,SC_127,SC_128
0,1,9,0,4,3,2.0,2.0,0,0,0,2,55,5,5,70.0,1,1,2,0,0.0,5020,autonomYRenta,4.260498e+09,3.669192e+09,5.913062e+08,9.026143e+08,2.0,58.0,0,0,1,1,0,0,1,0,1,0,1,0,1,2006,6.5,17.62,2.8,3.5,1.26,78.512833,684981607.0,NaN,319264308.0,NaN,3.428515e+07,NaN,NaN,0.0,108380640.0,22059893.0,44798551.0,NaN,NaN,293570921.0,NaN,59273497.0,1.510818e+08,23108834.0,187772652.0,NaN,NaN,2.873920e+07,285331628.0,13484075.0,3342355.0,194098171.0,96680199.0,NaN,126580873.0,67593206.0,2.429674e+08,NaN,NaN,NaN,18083827.0,NaN,1.077787e+07,NaN,414430200.0,NaN,NaN,238504987.0,NaN,NaN,NaN
1,4,9,1,5,3,1.0,1.0,1,1,0,2,49,4,5,65.0,1,1,2,0,0.0,1661,asalariado,1.157571e+09,1.157571e+09,0.000000e+00,4.608349e+08,1.0,36.0,0,1,0,1,0,0,0,0,1,1,1,0,1,2006,6.5,17.62,2.8,3.5,1.26,78.512833,242559505.0,63986399.0,4469436.0,3883917.0,1.340108e+08,1743799.0,131061450.0,NaN,NaN,5806520.0,24608584.0,409222.0,6257234.0,NaN,786645.0,220279.0,1.786810e+07,80340555.0,232260.0,NaN,NaN,1.025865e+07,12918413.0,NaN,NaN,52369740.0,1275591.0,NaN,180646.0,37937872.0,5.632546e+07,11337000.0,NaN,66360228.0,NaN,NaN,9.431210e+07,4.239700e+04,63833911.0,8.682130e+05,NaN,31037232.0,NaN,269128.0,NaN
2,5,9,0,1,1,1.0,1.0,2,2,0,4,38,5,1,120.0,1,1,3,0,0.0,1082,asalariado,1.935653e+09,1.532995e+09,4.026576e+08,8.357985e+08,1.0,72.0,1,1,1,1,0,0,1,1,0,2,1,0,0,2006,6.5,17.62,2.8,3.5,1.26,78.512833,192365971.0,14773914.0,3769746.0,114051264.0,1.549843e+08,133600548.0,7735605.0,0.0,2745595.0,41872899.0,51055630.0,7016520.0,2549481.0,4074811.0,31997196.0,980570.0,7.081891e+06,4270925.0,7888138.0,1133103.0,6.003756e+07,4.878288e+07,3965859.0,NaN,174323.0,39518923.0,170313451.0,326856.0,12485920.0,75591022.0,8.040671e+06,6319226.0,28630455.0,348646.0,610132.0,196114.0,3.662972e+07,1.418003e+08,53463727.0,3.769744e+06,2688584.0,50014460.0,NaN,5120752.0,217904.0
3,11,9,0,1,2,0.0,0.0,0,0,0,2,74,6,3,70.0,1,1,2,0,0.0,1282,pension,2.506370e+09,2.106291e+09,4.000792e+08,9.048945e+08,2.0,60.0,0,0,0,0,1,1,1,0,1,0,1,0,0,2006,6.5,17.62,2.8,3.5,1.26,78.512833,413683382.0,112061367.0,44211243.0,NaN,1.351971e+08,NaN,NaN,0.0,13747239.0,25970472.0,181376924.0,NaN,NaN,NaN,NaN,NaN,1.472135e+07,4600414.0,53275278.0,NaN,NaN,7.005875e+08,NaN,NaN,NaN,106605537.0,NaN,NaN,68746831.0,NaN,3.434982e+07,NaN,NaN,NaN,NaN,NaN,2.257742e+07,NaN,121190151.0,NaN,NaN,53388776.0,NaN,NaN,NaN
4,34,9,0,4,3,0.0,0.0,0,0,0,1,48,6,5,56.0,1,1,2,0,0.0,1685,pension,2.311958e+09,1.361849e+09,9.501089e+08,1.718298e+09,1.0,23.0,0,0,1,0,0,0,1,0,0,0,1,0,1,2006,6.5,17.62,2.8,3.5,1.26,78.512833,299567157.0,NaN,NaN,70557541.0,NaN,NaN,NaN,0.0,NaN,204843137.0,88457989.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,189679497.0,NaN,NaN,3449480.0,NaN,5.483329e+07,123396146.0,NaN,NaN,NaN,NaN,3.005975e+08,NaN,26467000.0,NaN,NaN,NaN,NaN,NaN,NaN
5,64,9,0,1,2,2.0,2.0,1,1,1,2,47,5,7,120.0,1,1,3,0,0.0,3000,autonomYRenta,4.038602e+09,3.121573e+09,9.170283e+08,1.207538e+09,2.0,42.0,1,0,1,1,0,0,1,1,1,0,1,0,0,2006,6.5,17.62,2.8,3.5,1.26,78.512833,663640314.0,76457071.0,152625367.0,49519917.0,1.578856e+07,5036640.0,NaN,0.0,202456508.0,84527644.0,289809064.0,43378379.0,NaN,NaN,NaN,2236360.0,6.875394e+07,5360260.0,623089.0,NaN,NaN,1.936625e+08,11780739.0,NaN,647240.0,67025591.0,9096382.0,NaN,19772223.0,105445820.0,3.293504e+08,NaN,NaN,NaN,NaN,31396.0,2.476829e+08,NaN,104571544.0

In [291]:
# Eliminar columnas NHIJOSD e NNINOSD
df_final.drop(columns=["NHIJOSD", "NNINOSD"], inplace=True, errors='ignore')

In [292]:
# COntar el número de valores -9 por columna
pd.set_option('display.max_rows', None)  # Mostrar todas las filas
missing_counts = (df_final == -9).sum()
print("Número de valores -9 por columna:")
print(missing_counts)

Número de valores -9 por columna:
NUMERO            0
CCAA              0
CAPROV            0
TAMAMU            0
DENSIDAD          0
NUMACTI           0
NUMOCU            0
NUMESTU          75
NUMNOESTU        75
EDADSP            0
REGTEN            0
ZONARES          12
SUPERF            0
AGUACALI          0
CALEF             2
TIPOCASA         14
DISPOSIOV         0
NUMOVD            0
IMPEXAC           0
FUENPRINRED       0
GASTOT            0
GASTMON           0
GASTNOMON         0
FACTOR            0
NUMPERI           0
COMITOT           0
NADUL_MAS         0
NADUL_FEM         0
NNINO_MAS         0
NNINO_FEM         0
ANC_MAS           0
ANC_FEM           0
PERCEP_MAS        0
NO_PERCEP_MAS     0
PERCEP_FEM        0
NO_PERCEP_FEM     0
NACION_ESP        0
NACION_EXT        0
EDUC_SUPERIOR     0
Año               0
Tasa_Paro         0
T_med             0
Inflacion         0
Tipo_Interes      0
EUR_USD           0
IPC               0
SC_11             0
SC_12             0
SC_21 

In [293]:
df_final = df_final[(df_final != -9).any(axis=1)]

In [294]:
#Borrar los nan que no sean outputs, los outputs son SC_
df_final = df_final.dropna(subset=[col for col in df_final.columns if not col.startswith('SC_')])


In [295]:
nan_counts = df_final.isna().sum()
nan_counts

NUMERO                0
CCAA                  0
CAPROV                0
TAMAMU                0
DENSIDAD              0
NUMACTI               0
NUMOCU                0
NUMESTU               0
NUMNOESTU             0
EDADSP                0
REGTEN                0
ZONARES               0
SUPERF                0
AGUACALI              0
CALEF                 0
TIPOCASA              0
DISPOSIOV             0
NUMOVD                0
IMPEXAC               0
FUENPRINRED           0
GASTOT                0
GASTMON               0
GASTNOMON             0
FACTOR                0
NUMPERI               0
COMITOT               0
NADUL_MAS             0
NADUL_FEM             0
NNINO_MAS             0
NNINO_FEM             0
ANC_MAS               0
ANC_FEM               0
PERCEP_MAS            0
NO_PERCEP_MAS         0
PERCEP_FEM            0
NO_PERCEP_FEM         0
NACION_ESP            0
NACION_EXT            0
EDUC_SUPERIOR         0
Año                   0
Tasa_Paro             0
T_med           

Creamos la variable de no ocupados y no activos

In [296]:
numero_miembros = (df_final["ANC_MAS"].astype(int) + df_final["ANC_FEM"].astype(int) +
    df_final["NADUL_MAS"].astype(int) + df_final["NADUL_FEM"].astype(int) +
    df_final["NNINO_MAS"].astype(int) + df_final["NNINO_FEM"].astype(int))

In [297]:
# Nueva variable no ocupados sera la resta entre ocupados y la suma de ANC_MAS, ANC_FEM, NADUL_MAS, NADUL_FEM, NNINO_MAS, NNINO_FEM
df_final["NUMNOOCU"] = numero_miembros - df_final["NUMOCU"].astype(int)

# Nueva variable no activos sera la resta entre activos y la suma de ANC_MAS, ANC_FEM, NADUL_MAS, NADUL_FEM, NNINO_MAS, NNINO_FEM
df_final["NUMNOACTI"] = numero_miembros -  df_final["NUMACTI"].astype(int)


In [298]:
numocu_counts = df_final["NUMOCU"].value_counts()
numacti_counts = df_final["NUMACTI"].value_counts()

print("Value counts de NUMOCU:")
print(numacti_counts)


Value counts de NUMOCU:
2.0    139420
1.0    107448
0.0     89637
3.0     22432
4.0      5748
5.0       537
6.0        80
7.0        13
8.0         3
Name: NUMACTI, dtype: int64


Gasto no monetario si es Nan es 0

In [299]:
# GASTNOMON si es NaN, se reemplaza por 0
df_final["GASTNOMON"] = df_final["GASTNOMON"].fillna(0)

Eliminamos la columna de Comunidad Autonoma ya que viene reflejada en la columna CCAA

In [300]:
df_final.drop(columns=['NACION_EXT'], inplace=True) 

In [301]:
df_final.head()  # Mostrar las primeras filas del DataFrame final

,NUMERO,CCAA,CAPROV,TAMAMU,DENSIDAD,NUMACTI,NUMOCU,NUMESTU,NUMNOESTU,EDADSP,REGTEN,ZONARES,SUPERF,AGUACALI,CALEF,TIPOCASA,DISPOSIOV,NUMOVD,IMPEXAC,FUENPRINRED,GASTOT,GASTMON,GASTNOMON,FACTOR,NUMPERI,COMITOT,NADUL_MAS,NADUL_FEM,NNINO_MAS,NNINO_FEM,ANC_MAS,ANC_FEM,PERCEP_MAS,NO_PERCEP_MAS,PERCEP_FEM,NO_PERCEP_FEM,NACION_ESP,EDUC_SUPERIOR,Año,Tasa_Paro,T_med,Inflacion,Tipo_Interes,EUR_USD,IPC,SC_11,SC_12,SC_21,SC_22,SC_31,SC_32,SC_41,SC_42,SC_43,SC_44,SC_45,SC_51,SC_52,SC_53,SC_54,SC_55,SC_56,SC_61,SC_62,SC_63,SC_71,SC_72,SC_73,SC_81,SC_82,SC_83,SC_91,SC_92,SC_93,SC_94,SC_95,SC_96,SC_101,SC_102,SC_103,SC_104,SC_111,SC_112,SC_121,SC_123,SC_124,SC_125,SC_126,SC_127,SC_128,NUMNOOCU,NUMNOACTI
0,1,9,0,4,3,2.0,2.0,0,2,55,5,5,70.0,1,1,2,0,0.0,5020,autonomYRenta,4.260498e+09,3.669192e+09,591306213.0,9.026143e+08,2.0,58.0,0,0,1,1,0,0,1,0,1,0,1,1,2006,6.5,17.62,2.8,3.5,1.26,78.512833,684981607.0,NaN,319264308.0,NaN,34285154.0,NaN,NaN,0.0,108380640.0,22059893.0,44798551.0,NaN,NaN,293570921.0,NaN,59273497.0,151081777.0,23108834.0,187772652.0,NaN,NaN,28739203.0,285331628.0,13484075.0,3342355.0,194098171.0,96680199.0,NaN,126580873.0,67593206.0,242967417.0,NaN,NaN,NaN,18083827.0,NaN,10777868.0,NaN,414430200.0,NaN,NaN,238504987.0,NaN,NaN,NaN,0,0
1,4,9,1,5,3,1.0,1.0,0,2,49,4,5,65.0,1,1,2,0,0.0,1661,asalariado,1.157571e+09,1.157571e+09,0.0,4.608349e+08,1.0,36.0,0,1,0,1,0,0,0,0,1,1,1,1,2006,6.5,17.62,2.8,3.5,1.26,78.512833,242559505.0,63986399.0,4469436.0,3883917.0,134010759.0,1743799.0,131061450.0,NaN,NaN,5806520.0,24608584.0,409222.0,6257234.0,NaN,786645.0,220279.0,17868098.0,80340555.0,232260.0,NaN,NaN,10258647.0,12918413.0,NaN,NaN,52369740.0,1275591.0,NaN,180646.0,37937872.0,56325460.0,11337000.0,NaN,66360228.0,NaN,NaN,94312103.0,42397.0,63833911.0,868213.0,NaN,31037232.0,NaN,269128.0,NaN,1,1
2,5,9,0,1,1,1.0,1.0,0,4,38,5,1,120.0,1,1,3,0,0.0,1082,asalariado,1.935653e+09,1.532995e+09,402657609.0,8.357985e+08,1.0,72.0,1,1,1,1,0,0,1,1,0,2,1,0,2006,6.5,17.62,2.8,3.5,1.26,78.512833,192365971.0,14773914.0,3769746.0,114051264.0,154984260.0,133600548.0,7735605.0,0.0,2745595.0,41872899.0,51055630.0,7016520.0,2549481.0,4074811.0,31997196.0,980570.0,7081891.0,4270925.0,7888138.0,1133103.0,60037560.0,48782882.0,3965859.0,NaN,174323.0,39518923.0,170313451.0,326856.0,12485920.0,75591022.0,8040671.0,6319226.0,28630455.0,348646.0,610132.0,196114.0,36629723.0,141800276.0,53463727.0,3769744.0,2688584.0,50014460.0,NaN,5120752.0,217904.0,3,3
3,11,9,0,1,2,0.0,0.0,0,2,74,6,3,70.0,1,1,2,0,0.0,1282,pension,2.506370e+09,2.106291e+09,400079199.0,9.048945e+08,2.0,60.0,0,0,0,0,1,1,1,0,1,0,1,0,2006,6.5,17.62,2.8,3.5,1.26,78.512833,413683382.0,112061367.0,44211243.0,NaN,135197087.0,NaN,NaN,0.0,13747239.0,25970472.0,181376924.0,NaN,NaN,NaN,NaN,NaN,14721353.0,4600414.0,53275278.0,NaN,NaN,700587473.0,NaN,NaN,NaN,106605537.0,NaN,NaN,68746831.0,NaN,34349824.0,NaN,NaN,NaN,NaN,NaN,22577416.0,NaN,121190151.0,NaN,NaN,53388776.0,NaN,NaN,NaN,2,2
4,34,9,0,4,3,0.0,0.0,0,1,48,6,5,56.0,1,1,2,0,0.0,1685,pension,2.311958e+09,1.361849e+09,950108912.0,1.718298e+09,1.0,23.0,0,0,1,0,0,0,1,0,0,0,1,1,2006,6.5,17.62,2.8,3.5,1.26,78.512833,299567157.0,NaN,NaN,70557541.0,NaN,NaN,NaN,0.0,NaN,204843137.0,88457989.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,189679497.0,NaN,NaN,3449480.0,NaN,54833289.0,123396146.0,NaN,NaN,NaN,NaN,300597524.0,NaN,26467000.0,NaN,NaN,NaN,NaN,NaN,NaN,1,1


### Dividir el GASTOT entre el factor poblacional

In [302]:
# El FACTOR de 2006 a 2015 hay que dividirlo por 1e6
df_final.loc[df_final['Año'].between(2006, 2015), 'FACTOR'] /= 1e6

# El GASTOMON  de 2006 a 2015 hay que dividirlo por 100
df_final.loc[df_final['Año'].between(2006, 2015), 'GASTNOMON'] /= 100
df_final.loc[df_final['Año'].between(2006, 2015), 'GASTMON'] /= 100

# Todas las supercategorías de 2006 a 2015 hay que dividirlas por 100
for col in df_final.columns:
    if col.startswith("SC_") and df_final['Año'].between(2006, 2015).any():
        df_final.loc[df_final['Año'].between(2006, 2015), col] /= 100


In [303]:
# Genero la lista de columnas a dividir
cols = ['GASTOT', 'GASTMON', 'GASTNOMON'] + [c for c in df_final.columns if c.startswith("SC_")]

# Divido y redondeo de un plumazo
df_final[cols] = df_final[cols].div(df_final['FACTOR'], axis=0).round(2)


### Elevar los ingresos mensuales a ingresos anuales

In [304]:
# Elevar IMPEXAC a Anual

df_final['IMPEXAC'] = df_final['IMPEXAC'] * 12  # Pasar de mensual a anual

In [305]:
df_final['tasaAhorro'] = (df_final['IMPEXAC'] - df_final['GASTMON']) / df_final['IMPEXAC']

In [306]:
# Elimina la columna factor
df_final.drop(columns=['FACTOR'], inplace=True)

## Eliminamos la categoria de gasto 42, ya que no refleja ningún gasto monetario

In [307]:
df_final.drop(columns=['SC_42', 'GASTOT', "GASTMON"], inplace=True)


## Eliminamos nulos de los inputs

In [308]:
output_columns = [col for col in df_final.columns if col.endswith(tuple(f"_{i}" for i in range(11, 129)))]

non_output_columns = [col for col in df_final.columns if col not in output_columns]
df_final = df_final.dropna(subset=non_output_columns)

### Calculamos la inflación acumulada y la aplicamos a todos las columnas que sean tipo €

In [309]:
inflacion_df = pd.read_csv('../../data/inflacion_espana_2006_2023.csv', sep='\t')

In [310]:
codigos_comunidad = {
    "Andalucía": 1, "Aragón": 2, "Asturias": 3, "Islas Baleares": 4,
    "Canarias": 5, "Cantabria": 6, "Castilla y León": 8, "Castilla-La Mancha": 7,
    "Cataluña": 9, "Comunidad Valenciana": 10, "Extremadura": 11, "Galicia": 12,
    "Madrid": 13, "Murcia": 14, "Navarra": 15,
    "País Vasco": 16, "La Rioja": 17, "Ceuta": 18, "Melilla": 19
}

# Agregar código de comunidad
inflacion_df["CCAA"] = inflacion_df["Comunidad Autónoma"].map(codigos_comunidad)



In [311]:
inflacion_df.head()

,Año,Comunidad Autónoma,Inflación (%),CCAA
0,2006,Andalucía,2.9,1
1,2006,Aragón,2.9,2
2,2006,Asturias,2.4,3
3,2006,Islas Baleares,2.8,4
4,2006,Canarias,2.0,5


In [312]:
def calcular_factores_acumulados_a_2023(df_inflacion):
    """
    Dado un DataFrame con columnas: 'Año', 'Comunidad Autónoma', 'Inflación (%)', 'CCAA',
    devuelve un DataFrame con la inflación acumulada para ajustar precios a 2023.
    """
    df = df_inflacion.copy()
    df = df.sort_values(['CCAA', 'Año'])

    # A porcentaje decimal
    df['inflacion_decimal'] = df['Inflación (%)'] / 100

    # Calcular factor acumulado por CCAA
    df['factor_acumulado'] = df.groupby('CCAA')['inflacion_decimal'].transform(lambda x: (1 + x).cumprod())

    # Obtener el factor en 2023 para cada CCAA
    factores_2023 = df[df['Año'] == 2023].set_index('CCAA')['factor_acumulado'].to_dict()

    # Crear factor de ajuste a 2023: factor_acumulado / factor_2023
    df['factor_ajuste_2023'] = df.apply(
        lambda row: row['factor_acumulado'] / factores_2023.get(row['CCAA'], np.nan),
        axis=1
    )

    return df[['Año', 'Comunidad Autónoma', 'CCAA', 'factor_ajuste_2023']]


In [313]:
inflacion_ajuste = calcular_factores_acumulados_a_2023(inflacion_df)
# Eliminar filas donde Comunidad Autónoma es Nacional
inflacion_ajuste = inflacion_ajuste[inflacion_ajuste['Comunidad Autónoma'] != 'Nacional']

inflacion_ajuste.head()

,Año,Comunidad Autónoma,CCAA,factor_ajuste_2023
0,2006,Andalucía,1,0.722433
19,2007,Andalucía,1,0.752775
38,2008,Andalucía,1,0.762561
57,2009,Andalucía,1,0.767136
76,2010,Andalucía,1,0.791685


In [314]:
def aplicar_ajuste_inflacion(df_final, df_factores):
    """
    Aplica los factores de ajuste a 2023 a las columnas monetarias del df_final,
    modificando directamente las columnas que comienzan por 'SC_' más 'GASTOT' e 'IMPEXAC'.
    """
    df = df_final.copy()

    # Unir el factor a cada fila según Año y CCAA
    df = df.merge(df_factores[['Año', 'CCAA', 'factor_ajuste_2023']],
                  on=['Año', 'CCAA'], how='left')

    # Columnas monetarias a ajustar
    columnas_monetarias = [col for col in df.columns if col.startswith('SC_')]
    columnas_monetarias += [col for col in ['GASTOT', 'GASTMON', 'IMPEXAC', "GASTNOMON"] if col in df.columns]

    # Ajustar directamente las columnas (manteniendo NaNs)
    for col in columnas_monetarias:
        df[col] = df[col] * df['factor_ajuste_2023']

    # Eliminar columna auxiliar
    df.drop(columns=['factor_ajuste_2023'], inplace=True)

    return df


In [315]:
df_final_ajustado = aplicar_ajuste_inflacion(df_final, inflacion_ajuste)

pd.set_option('display.max_columns', None)  # Mostrar todas las columnas en el output
df_final_ajustado.head(5)  # Mostrar las primeras filas del DataFrame ajustado

,NUMERO,CCAA,CAPROV,TAMAMU,DENSIDAD,NUMACTI,NUMOCU,NUMESTU,NUMNOESTU,EDADSP,REGTEN,ZONARES,SUPERF,AGUACALI,CALEF,TIPOCASA,DISPOSIOV,NUMOVD,IMPEXAC,FUENPRINRED,GASTNOMON,NUMPERI,COMITOT,NADUL_MAS,NADUL_FEM,NNINO_MAS,NNINO_FEM,ANC_MAS,ANC_FEM,PERCEP_MAS,NO_PERCEP_MAS,PERCEP_FEM,NO_PERCEP_FEM,NACION_ESP,EDUC_SUPERIOR,Año,Tasa_Paro,T_med,Inflacion,Tipo_Interes,EUR_USD,IPC,SC_11,SC_12,SC_21,SC_22,SC_31,SC_32,SC_41,SC_43,SC_44,SC_45,SC_51,SC_52,SC_53,SC_54,SC_55,SC_56,SC_61,SC_62,SC_63,SC_71,SC_72,SC_73,SC_81,SC_82,SC_83,SC_91,SC_92,SC_93,SC_94,SC_95,SC_96,SC_101,SC_102,SC_103,SC_104,SC_111,SC_112,SC_121,SC_123,SC_124,SC_125,SC_126,SC_127,SC_128,NUMNOOCU,NUMNOACTI,tasaAhorro
0,1,9,0,4,3,2.0,2.0,0,2,55,5,5,70.0,1,1,2,0,0.0,42599.711960,autonomYRenta,4632.676246,2.0,58.0,0,0,1,1,0,0,1,0,1,0,1,1,2006,6.5,17.62,2.8,3.5,1.26,78.512833,5366.587817,NaN,2501.325816,NaN,268.610136,NaN,NaN,849.123143,172.831501,350.980894,NaN,NaN,2300.023791,NaN,464.389191,1183.669487,181.048776,1471.132682,NaN,NaN,225.161824,2235.473630,105.643608,26.186377,1520.690913,757.453145,NaN,991.716203,529.568730,1903.565017,NaN,NaN,NaN,141.680815,NaN,84.442756,NaN,3246.912707,NaN,NaN,1868.602704,NaN,NaN,NaN,0,0,0.325187
1,4,9,1,5,3,1.0,1.0,0,2,49,4,5,65.0,1,1,2,0,0.0,14095.243340,asalariado,0.000000,1.0,36.0,0,1,0,1,0,0,0,0,1,1,1,1,2006,6.5,17.62,2.8,3.5,1.26,78.512833,3722.156904,981.893660,68.588082,59.599995,2056.440279,26.759182,2011.181620,NaN,89.102983,377.626929,6.279639,96.019072,NaN,12.071333,3.380256,274.189680,1232.852919,3.564119,NaN,NaN,157.422342,198.239994,NaN,NaN,803.631120,19.574370,NaN,2.772093,582.167777,864.334295,173.970039,NaN,1018.319808,NaN,NaN,1447.251669,0.650593,979.552938,13.323017,NaN,476.276660,NaN,4.129853,NaN,1,1,-0.260235
2,5,9,0,1,1,1.0,1.0,0,4,38,5,1,120.0,1,1,3,0,0.0,9181.850267,asalariado,3406.873777,1.0,72.0,1,1,1,1,0,0,1,1,0,2,1,0,2006,6.5,17.62,2.8,3.5,1.26,78.512833,1627.600349,124.998756,31.893211,964.985308,1311.320118,1130.391560,65.448263,23.230421,354.283361,431.979748,59.366630,21.568579,34.474368,270.724564,8.295063,59.918220,36.136210,66.742377,9.589178,507.978936,412.751890,33.555052,NaN,1.477978,334.369552,1441.014460,2.765021,105.643608,639.575556,68.029420,53.468862,242.239896,2.948884,5.162316,1.661841,309.922805,1199.764597,452.353216,31.893211,22.749547,423.168453,NaN,43.328093,1.845705,3,3,-0.412638
3,11,9,0,1,2,0.0,0.0,0,2,74,6,3,70.0,1,1,2,0,0.0,10879.049947,pension,3126.581250,2.0,60.0,0,0,0,0,1,1,1,0,1,0,1,0,2006,6.5,17.62,2.8,3.5,1.26,78.512833,3232.896667,875.747963,345.507425,NaN,1056.549231,NaN,NaN,107.432740,202.956795,1417.444599,NaN,NaN,NaN,NaN,NaN,115.048923,35.952347,416.344296,NaN,NaN,5475.024733,NaN,NaN,NaN,833.112893,NaN,NaN,537.248559,NaN,268.440416,NaN,NaN,NaN,NaN,NaN,176.438050,NaN,947.086923,NaN,NaN,417.228255,NaN,NaN,NaN,2,2,-0.513042
4,34,9,0,4,3,0.0,0.0,0,1,48,6,5,56.0,1,1,2,0,0.0,14298.907301,pension,3910.178342,1.0,23.0,0,0,1,0,0,0,1,0,0,0,1,1,2006,6.5,17.62,2.8,3.5,1.26,78.512833,1232.867062,NaN,NaN,290.376722,NaN,NaN,NaN,NaN,843.034439,364.049331,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,780.626993,NaN,NaN,14.192832,NaN,225.663912,507.837502,NaN,NaN,NaN,NaN,1237.110062,NaN,108.924861,NaN,NaN,NaN,NaN,NaN,NaN,1,1,0.608033


Renombrar tasaAhorro

In [316]:
# Renombrar tasaAhorro a tasa_ahorro
df_final_ajustado.rename(columns={'tasaAhorro': 'tasa_ahorro'}, inplace=True)

Pasamos el año a timestamp

In [317]:
# 1) Crear datetime en UTC a partir del año
df_final_ajustado['Timestamp'] = pd.to_datetime(
    df_final_ajustado['Año'].astype(str) + '-01-01', utc=True
)

# 2) Formatear como ISO 8601 con 'Z'
df_final_ajustado['Timestamp'] = df_final_ajustado['Timestamp'].dt.strftime('%Y-%m-%dT%H:%M:%SZ')

# 3) Eliminar la columna 'Año' si ya no hace falta
df_final_ajustado.drop(columns=['Año'], inplace=True)

df_final_ajustado['Timestamp'].head()


0    2006-01-01T00:00:00Z
1    2006-01-01T00:00:00Z
2    2006-01-01T00:00:00Z
3    2006-01-01T00:00:00Z
4    2006-01-01T00:00:00Z
Name: Timestamp, dtype: object

In [318]:
df_final_ajustado.head()  # Mostrar las primeras filas de la columna GASTOT ajustada

,NUMERO,CCAA,CAPROV,TAMAMU,DENSIDAD,NUMACTI,NUMOCU,NUMESTU,NUMNOESTU,EDADSP,REGTEN,ZONARES,SUPERF,AGUACALI,CALEF,TIPOCASA,DISPOSIOV,NUMOVD,IMPEXAC,FUENPRINRED,GASTNOMON,NUMPERI,COMITOT,NADUL_MAS,NADUL_FEM,NNINO_MAS,NNINO_FEM,ANC_MAS,ANC_FEM,PERCEP_MAS,NO_PERCEP_MAS,PERCEP_FEM,NO_PERCEP_FEM,NACION_ESP,EDUC_SUPERIOR,Tasa_Paro,T_med,Inflacion,Tipo_Interes,EUR_USD,IPC,SC_11,SC_12,SC_21,SC_22,SC_31,SC_32,SC_41,SC_43,SC_44,SC_45,SC_51,SC_52,SC_53,SC_54,SC_55,SC_56,SC_61,SC_62,SC_63,SC_71,SC_72,SC_73,SC_81,SC_82,SC_83,SC_91,SC_92,SC_93,SC_94,SC_95,SC_96,SC_101,SC_102,SC_103,SC_104,SC_111,SC_112,SC_121,SC_123,SC_124,SC_125,SC_126,SC_127,SC_128,NUMNOOCU,NUMNOACTI,tasa_ahorro,Timestamp
0,1,9,0,4,3,2.0,2.0,0,2,55,5,5,70.0,1,1,2,0,0.0,42599.711960,autonomYRenta,4632.676246,2.0,58.0,0,0,1,1,0,0,1,0,1,0,1,1,6.5,17.62,2.8,3.5,1.26,78.512833,5366.587817,NaN,2501.325816,NaN,268.610136,NaN,NaN,849.123143,172.831501,350.980894,NaN,NaN,2300.023791,NaN,464.389191,1183.669487,181.048776,1471.132682,NaN,NaN,225.161824,2235.473630,105.643608,26.186377,1520.690913,757.453145,NaN,991.716203,529.568730,1903.565017,NaN,NaN,NaN,141.680815,NaN,84.442756,NaN,3246.912707,NaN,NaN,1868.602704,NaN,NaN,NaN,0,0,0.325187,2006-01-01T00:00:00Z
1,4,9,1,5,3,1.0,1.0,0,2,49,4,5,65.0,1,1,2,0,0.0,14095.243340,asalariado,0.000000,1.0,36.0,0,1,0,1,0,0,0,0,1,1,1,1,6.5,17.62,2.8,3.5,1.26,78.512833,3722.156904,981.893660,68.588082,59.599995,2056.440279,26.759182,2011.181620,NaN,89.102983,377.626929,6.279639,96.019072,NaN,12.071333,3.380256,274.189680,1232.852919,3.564119,NaN,NaN,157.422342,198.239994,NaN,NaN,803.631120,19.574370,NaN,2.772093,582.167777,864.334295,173.970039,NaN,1018.319808,NaN,NaN,1447.251669,0.650593,979.552938,13.323017,NaN,476.276660,NaN,4.129853,NaN,1,1,-0.260235,2006-01-01T00:00:00Z
2,5,9,0,1,1,1.0,1.0,0,4,38,5,1,120.0,1,1,3,0,0.0,9181.850267,asalariado,3406.873777,1.0,72.0,1,1,1,1,0,0,1,1,0,2,1,0,6.5,17.62,2.8,3.5,1.26,78.512833,1627.600349,124.998756,31.893211,964.985308,1311.320118,1130.391560,65.448263,23.230421,354.283361,431.979748,59.366630,21.568579,34.474368,270.724564,8.295063,59.918220,36.136210,66.742377,9.589178,507.978936,412.751890,33.555052,NaN,1.477978,334.369552,1441.014460,2.765021,105.643608,639.575556,68.029420,53.468862,242.239896,2.948884,5.162316,1.661841,309.922805,1199.764597,452.353216,31.893211,22.749547,423.168453,NaN,43.328093,1.845705,3,3,-0.412638,2006-01-01T00:00:00Z
3,11,9,0,1,2,0.0,0.0,0,2,74,6,3,70.0,1,1,2,0,0.0,10879.049947,pension,3126.581250,2.0,60.0,0,0,0,0,1,1,1,0,1,0,1,0,6.5,17.62,2.8,3.5,1.26,78.512833,3232.896667,875.747963,345.507425,NaN,1056.549231,NaN,NaN,107.432740,202.956795,1417.444599,NaN,NaN,NaN,NaN,NaN,115.048923,35.952347,416.344296,NaN,NaN,5475.024733,NaN,NaN,NaN,833.112893,NaN,NaN,537.248559,NaN,268.440416,NaN,NaN,NaN,NaN,NaN,176.438050,NaN,947.086923,NaN,NaN,417.228255,NaN,NaN,NaN,2,2,-0.513042,2006-01-01T00:00:00Z
4,34,9,0,4,3,0.0,0.0,0,1,48,6,5,56.0,1,1,2,0,0.0,14298.907301,pension,3910.178342,1.0,23.0,0,0,1,0,0,0,1,0,0,0,1,1,6.5,17.62,2.8,3.5,1.26,78.512833,1232.867062,NaN,NaN,290.376722,NaN,NaN,NaN,NaN,843.034439,364.049331,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,780.626993,NaN,NaN,14.192832,NaN,225.663912,507.837502,NaN,NaN,NaN,NaN,1237.110062,NaN,108.924861,NaN,NaN,NaN,NaN,NaN,NaN,1,1,0.608033,2006-01-01T00:00:00Z


In [319]:
df_final_ajustado.tail()

,NUMERO,CCAA,CAPROV,TAMAMU,DENSIDAD,NUMACTI,NUMOCU,NUMESTU,NUMNOESTU,EDADSP,REGTEN,ZONARES,SUPERF,AGUACALI,CALEF,TIPOCASA,DISPOSIOV,NUMOVD,IMPEXAC,FUENPRINRED,GASTNOMON,NUMPERI,COMITOT,NADUL_MAS,NADUL_FEM,NNINO_MAS,NNINO_FEM,ANC_MAS,ANC_FEM,PERCEP_MAS,NO_PERCEP_MAS,PERCEP_FEM,NO_PERCEP_FEM,NACION_ESP,EDUC_SUPERIOR,Tasa_Paro,T_med,Inflacion,Tipo_Interes,EUR_USD,IPC,SC_11,SC_12,SC_21,SC_22,SC_31,SC_32,SC_41,SC_43,SC_44,SC_45,SC_51,SC_52,SC_53,SC_54,SC_55,SC_56,SC_61,SC_62,SC_63,SC_71,SC_72,SC_73,SC_81,SC_82,SC_83,SC_91,SC_92,SC_93,SC_94,SC_95,SC_96,SC_101,SC_102,SC_103,SC_104,SC_111,SC_112,SC_121,SC_123,SC_124,SC_125,SC_126,SC_127,SC_128,NUMNOOCU,NUMNOACTI,tasa_ahorro,Timestamp
365313,19940,18,1,4,3,4.0,0.0,0,8,43,3,4,82.0,1,0,2,0,0.0,12960.0,pension,0.0,2.0,112.0,2,2,4,0,0,0,1,5,1,1,0,0,30.0,19.6,3.3,4.5,1.08,112.407167,5646.01,1503.59,NaN,NaN,8760.02,1460.00,1176.0,NaN,312.0,540.0,NaN,NaN,NaN,NaN,NaN,216.69,NaN,262.75,NaN,NaN,1298.41,NaN,NaN,NaN,360.0,NaN,NaN,NaN,NaN,660.36,NaN,NaN,NaN,NaN,NaN,375.04,NaN,NaN,NaN,NaN,NaN,320.0,NaN,NaN,8,4,-0.766272,2023-01-01T00:00:00Z
365314,20356,18,1,4,3,1.0,0.0,0,3,85,3,6,200.0,1,0,3,0,0.0,44160.0,pension,0.0,2.0,34.0,1,0,0,0,1,1,2,0,0,1,1,1,30.0,19.6,3.3,4.5,1.08,112.407167,10612.64,4202.06,1360.66,1132.79,216.69,193.80,2880.0,NaN,540.0,1260.0,NaN,NaN,129.2,NaN,NaN,351.41,3278.62,516.81,NaN,NaN,NaN,NaN,NaN,NaN,672.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9920.55,NaN,2738.96,NaN,NaN,NaN,NaN,NaN,NaN,3,2,0.094063,2023-01-01T00:00:00Z
365315,20450,18,1,4,3,2.0,0.0,0,3,30,3,5,70.0,1,0,2,0,0.0,5472.0,pension,0.0,1.0,42.0,1,1,0,1,0,0,1,0,0,2,1,0,30.0,19.6,3.3,4.5,1.08,112.407167,2778.56,NaN,NaN,NaN,NaN,NaN,NaN,NaN,456.0,726.6,NaN,NaN,NaN,NaN,NaN,NaN,1401.36,NaN,NaN,NaN,NaN,255.50,NaN,NaN,96.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,1,-0.044229,2023-01-01T00:00:00Z
365316,20607,18,1,4,3,2.0,2.0,1,3,64,6,6,128.0,1,0,2,0,0.0,96000.0,asalariado,9531.9,2.0,48.0,1,1,2,0,0,0,1,2,1,0,1,1,30.0,19.6,3.3,4.5,1.08,112.407167,8035.61,835.01,232.56,NaN,9879.36,1216.67,NaN,NaN,2400.0,2160.0,NaN,NaN,1967.6,NaN,NaN,1142.88,1313.77,NaN,NaN,NaN,NaN,851.67,NaN,NaN,1920.0,NaN,NaN,NaN,NaN,218.96,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1824.81,NaN,NaN,NaN,800.0,NaN,NaN,2,2,0.637511,2023-01-01T00:00:00Z
365317,20639,18,1,4,3,1.0,1.0,0,1,47,4,5,45.0,1,0,1,0,0.0,39600.0,asalariado,1356.0,1.0,0.0,1,0,0,0,0,0,1,0,0,0,1,0,30.0,19.6,3.3,4.5,1.08,112.407167,1998.93,44.30,NaN,NaN,1825.00,NaN,7200.0,NaN,504.0,360.0,NaN,NaN,NaN,NaN,NaN,121.67,664.47,1423.25,NaN,NaN,NaN,389.33,NaN,NaN,240.0,NaN,NaN,NaN,NaN,48.67,NaN,NaN,NaN,NaN,NaN,9795.34,NaN,NaN,NaN,437.92,NaN,800.0,NaN,NaN,0,0,0.347149,2023-01-01T00:00:00Z


Borramos la columna numero ya que el identificador no va a ser necesario para entrenar nuestro modelo

In [320]:
df_final_ajustado.drop(columns=['NUMERO'], inplace=True)

### Renombramos las columnas de gasto mensual por categorías para que se entienda

In [321]:
def renombrar_columnas_sc(df, categoria_detallada):
    """
    Renombra las columnas SC_{codigo} a una abreviatura legible basada en el nombre de la categoría.
    Ejemplo: SC_011 → ALIMENTOS_011
    """
    df = df.copy()
    columnas_renombradas = {}

    for col in df.columns:
        if col.startswith("SC_"):
            codigo = col.split("_")[1]
            nombre_categoria = categoria_detallada.get(codigo.lstrip("0"), None)

            if nombre_categoria:
                # Tomamos una abreviatura clara: primeras palabras en mayúsculas, separadas por _
                abreviatura = (
                    nombre_categoria.upper()
                    .replace(",", "")
                    .replace(".", "")
                    .replace("/", "_")
                    .replace(" Y ", "_")
                    .replace(" ", "_")
                )
                nuevo_nombre = f"{abreviatura}_{codigo}"
                columnas_renombradas[col] = nuevo_nombre

    return df.rename(columns=columnas_renombradas)

In [322]:
# Diccionario de nombres sin puntos
categoria_detallada = {
        "11": "PRODUCTOS ALIMENTICIOS",
        "12": "BEBIDAS NO ALCOHÓLICAS",
        "21": "BEBIDAS ALCOHÓLICAS",
        "22": "TABACO",
        "31": "ARTÍCULOS DE VESTIR",
        "32": "CALZADO",
        "41": "ALQUILERES REALES DE LA VIVIENDA",
        "43": "MANTENIMIENTO DE LA VIVIENDA",
        "44": "SUMINISTRO DE AGUA",
        "45": "ELECTRICIDAD, GAS Y OTROS COMBUSTIBLES",
        "51": "MUEBLES Y REVESTIMIENTOS",
        "52": "TEXTILES PARA EL HOGAR",
        "53": "GRANDES ELECTRODOMÉSTICOS",
        "54": "UTENSILIOS DEL HOGAR",
        "55": "HERRAMIENTAS CASA/JARDÍN",
        "56": "BIENES Y SERVICIOS PARA EL HOGAR",
        "61": "PRODUCTOS FARMACÉUTICOS",
        "62": "SERVICIOS MÉDICOS AMBULATORIOS",
        "63": "SERVICIOS HOSPITALARIOS",
        "71": "COMPRA DE VEHÍCULOS",
        "72": "USO DE VEHÍCULOS PERSONALES",
        "73": "SERVICIOS DE TRANSPORTE",
        "81": "SERVICIOS POSTALES",
        "82": "EQUIPOS TELÉFONO/FAX",
        "83": "SERVICIOS TELÉFONO/FAX",
        "91": "AUDIOVISUALES Y TECNOLOGÍA",
        "92": "BIENES DURADEROS DE OCIO",
        "93": "OCIO, JARDINERÍA Y MASCOTAS",
        "94": "SERVICIOS RECREATIVOS Y CULTURALES",
        "95": "PRENSA Y PAPELERÍA",
        "96": "PAQUETES TURÍSTICOS",
        "101": "EDUCACIÓN INFANTIL Y PRIMARIA",
        "102": "EDUCACIÓN SECUNDARIA Y POSTSECUNDARIA",
        "103": "EDUCACIÓN SUPERIOR",
        "104": "EDUCACIÓN NO FORMAL",
        "111": "RESTAURACIÓN",
        "112": "ALOJAMIENTO",
        "121": "CUIDADOS PERSONALES",
        "123": "EFECTOS PERSONALES",
        "124": "PROTECCIÓN SOCIAL",
        "125": "SEGUROS",
        "126": "SERVICIOS FINANCIEROS",
        "127": "OTROS SERVICIOS",
        "128": "REMESAS"
    }

df_final_ajustado = renombrar_columnas_sc(df_final_ajustado, categoria_detallada)

In [323]:

pd.set_option('display.max_columns', None)  # Mostrar todas las columnas en el outpu
 # No Mostrar las filas en el output
df_final_ajustado.head(5)  # Mostrar las primeras filas del DataFrame ajustado

,CCAA,CAPROV,TAMAMU,DENSIDAD,NUMACTI,NUMOCU,NUMESTU,NUMNOESTU,EDADSP,REGTEN,ZONARES,SUPERF,AGUACALI,CALEF,TIPOCASA,DISPOSIOV,NUMOVD,IMPEXAC,FUENPRINRED,GASTNOMON,NUMPERI,COMITOT,NADUL_MAS,NADUL_FEM,NNINO_MAS,NNINO_FEM,ANC_MAS,ANC_FEM,PERCEP_MAS,NO_PERCEP_MAS,PERCEP_FEM,NO_PERCEP_FEM,NACION_ESP,EDUC_SUPERIOR,Tasa_Paro,T_med,Inflacion,Tipo_Interes,EUR_USD,IPC,PRODUCTOS_ALIMENTICIOS_11,BEBIDAS_NO_ALCOHÓLICAS_12,BEBIDAS_ALCOHÓLICAS_21,TABACO_22,ARTÍCULOS_DE_VESTIR_31,CALZADO_32,ALQUILERES_REALES_DE_LA_VIVIENDA_41,MANTENIMIENTO_DE_LA_VIVIENDA_43,SUMINISTRO_DE_AGUA_44,ELECTRICIDAD_GAS_OTROS_COMBUSTIBLES_45,MUEBLES_REVESTIMIENTOS_51,TEXTILES_PARA_EL_HOGAR_52,GRANDES_ELECTRODOMÉSTICOS_53,UTENSILIOS_DEL_HOGAR_54,HERRAMIENTAS_CASA_JARDÍN_55,BIENES_SERVICIOS_PARA_EL_HOGAR_56,PRODUCTOS_FARMACÉUTICOS_61,SERVICIOS_MÉDICOS_AMBULATORIOS_62,SERVICIOS_HOSPITALARIOS_63,COMPRA_DE_VEHÍCULOS_71,USO_DE_VEHÍCULOS_PERSONALES_72,SERVICIOS_DE_TRANSPORTE_73,SERVICIOS_POSTALES_81,EQUIPOS_TELÉFONO_FAX_82,SERVICIOS_TELÉFONO_FAX_83,AUDIOVISUALES_TECNOLOGÍA_91,BIENES_DURADEROS_DE_OCIO_92,OCIO_JARDINERÍA_MASCOTAS_93,SERVICIOS_RECREATIVOS_CULTURALES_94,PRENSA_PAPELERÍA_95,PAQUETES_TURÍSTICOS_96,EDUCACIÓN_INFANTIL_PRIMARIA_101,EDUCACIÓN_SECUNDARIA_POSTSECUNDARIA_102,EDUCACIÓN_SUPERIOR_103,EDUCACIÓN_NO_FORMAL_104,RESTAURACIÓN_111,ALOJAMIENTO_112,CUIDADOS_PERSONALES_121,EFECTOS_PERSONALES_123,PROTECCIÓN_SOCIAL_124,SEGUROS_125,SERVICIOS_FINANCIEROS_126,OTROS_SERVICIOS_127,REMESAS_128,NUMNOOCU,NUMNOACTI,tasa_ahorro,Timestamp
0,9,0,4,3,2.0,2.0,0,2,55,5,5,70.0,1,1,2,0,0.0,42599.711960,autonomYRenta,4632.676246,2.0,58.0,0,0,1,1,0,0,1,0,1,0,1,1,6.5,17.62,2.8,3.5,1.26,78.512833,5366.587817,NaN,2501.325816,NaN,268.610136,NaN,NaN,849.123143,172.831501,350.980894,NaN,NaN,2300.023791,NaN,464.389191,1183.669487,181.048776,1471.132682,NaN,NaN,225.161824,2235.473630,105.643608,26.186377,1520.690913,757.453145,NaN,991.716203,529.568730,1903.565017,NaN,NaN,NaN,141.680815,NaN,84.442756,NaN,3246.912707,NaN,NaN,1868.602704,NaN,NaN,NaN,0,0,0.325187,2006-01-01T00:00:00Z
1,9,1,5,3,1.0,1.0,0,2,49,4,5,65.0,1,1,2,0,0.0,14095.243340,asalariado,0.000000,1.0,36.0,0,1,0,1,0,0,0,0,1,1,1,1,6.5,17.62,2.8,3.5,1.26,78.512833,3722.156904,981.893660,68.588082,59.599995,2056.440279,26.759182,2011.181620,NaN,89.102983,377.626929,6.279639,96.019072,NaN,12.071333,3.380256,274.189680,1232.852919,3.564119,NaN,NaN,157.422342,198.239994,NaN,NaN,803.631120,19.574370,NaN,2.772093,582.167777,864.334295,173.970039,NaN,1018.319808,NaN,NaN,1447.251669,0.650593,979.552938,13.323017,NaN,476.276660,NaN,4.129853,NaN,1,1,-0.260235,2006-01-01T00:00:00Z
2,9,0,1,1,1.0,1.0,0,4,38,5,1,120.0,1,1,3,0,0.0,9181.850267,asalariado,3406.873777,1.0,72.0,1,1,1,1,0,0,1,1,0,2,1,0,6.5,17.62,2.8,3.5,1.26,78.512833,1627.600349,124.998756,31.893211,964.985308,1311.320118,1130.391560,65.448263,23.230421,354.283361,431.979748,59.366630,21.568579,34.474368,270.724564,8.295063,59.918220,36.136210,66.742377,9.589178,507.978936,412.751890,33.555052,NaN,1.477978,334.369552,1441.014460,2.765021,105.643608,639.575556,68.029420,53.468862,242.239896,2.948884,5.162316,1.661841,309.922805,1199.764597,452.353216,31.893211,22.749547,423.168453,NaN,43.328093,1.845705,3,3,-0.412638,2006-01-01T00:00:00Z
3,9,0,1,2,0.0,0.0,0,2,74,6,3,70.0,1,1,2,0,0.0,10879.049947,pension,3126.581250,2.0,60.0,0,0,0,0,1,1,1,0,1,0,1,0,6.5,17.62,2.8,3.5,1.26,78.512833,3232.896667,875.747963,345.507425,NaN,1056.549231,NaN,NaN,107.432740,202.956795,1417.444599,NaN,NaN,NaN,NaN,NaN,115.048923,35.952347,416.344296,NaN,NaN,5475.024733,NaN,NaN,NaN,833.112893,NaN,NaN,537.248559,NaN,268.440416,NaN,NaN,NaN,NaN,NaN,176.438050,NaN,947.086923,NaN,NaN,417.228255,NaN,NaN,NaN,2,2,-0.513042,2006-01-01T00:00:00Z
4,9,0,4,3,0.0,0.0,0,1,48,6,5,56.0,1,1,2,0,0.0,14298.907301,pension,3910.178342,1.0,23.0,0,0,1,0,0,0,1,0,0,0,1,1,6.5,17.62,2.8,3.5,1.26,78.512833,1232.867062,NaN,NaN,290.376722,NaN,NaN,NaN,NaN,843.034439,364.049331,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,780.626993,NaN,NaN

Eliminamos NNINOSD, NUMPERI, DISPOSIOV y NHIJOSD ya que esta repetida con las añadidas

In [324]:
df_final_ajustado.drop(columns=['NUMPERI', 'DISPOSIOV'], inplace=True)


### Ordenamos las columnas de nuestro dataframe

In [325]:
columnas_prioritarias = [
    # Variables de identificación y ponderación
    "Timestamp",
    
    "CCAA",        
    "CAPROV",
    "TAMAMU",
    "DENSIDAD",
    "SUPERF",
    "TIPOCASA",
    "AGUACALI",
    "CALEF",
    "ZONARES",
    "REGTEN",
    "COMITOT",

    # Composición del hogar
    "NUMESTU",
    "NUMNOESTU",
    "NUMOCU",
    "NUMNOOCU",
    "NUMACTI",
    "NUMNOACTI",
    "ANC_MAS",
    "ANC_FEM",
    "NADUL_MAS",
    "NADUL_FEM",
    "NNINO_FEM",
    "NNINO_MAS",
    "PERCEP_MAS",
    "PERCEP_FEM",
    "NO_PERCEP_MAS",
    "NO_PERCEP_FEM",

    # Datos del sustentador principal
    "OCUSP",
    "EDADSP",
    "NACION_ESP",
    "EDUC_SUPERIOR",

    # Características de propiedad e ingresos
    "FUENPRINRED",
    "NUMOVD",
    "IMPEXAC",
    "GASTNOMON",
    "tasa_ahorro",
    
    # Variables ambientales y macro
    "T_med",
    "Tasa_Paro",
    "Inflacion",
    "Tipo_Interes",
    "EUR_USD",
    "IPC",

    # --- A partir de aquí, todos los outputs (gasto por categoría) ---
    "PRODUCTOS_ALIMENTICIOS_11",
    "BEBIDAS_NO_ALCOHÓLICAS_12",
    "BEBIDAS_ALCOHÓLICAS_21",
    "TABACO_22",
    "ARTÍCULOS_DE_VESTIR_31",
    "CALZADO_32",
    "ALQUILERES_REALES_DE_LA_VIVIENDA_41",
    "MANTENIMIENTO_DE_LA_VIVIENDA_43",
    "SUMINISTRO_DE_AGUA_44",
    "ELECTRICIDAD_GAS_OTROS_COMBUSTIBLES_45",
    "MUEBLES_REVESTIMIENTOS_51",
    "TEXTILES_PARA_EL_HOGAR_52",
    "GRANDES_ELECTRODOMÉSTICOS_53",
    "UTENSILIOS_DEL_HOGAR_54",
    "HERRAMIENTAS_CASA_JARDÍN_55",
    "BIENES_SERVICIOS_PARA_EL_HOGAR_56",
    "PRODUCTOS_FARMACÉUTICOS_61",
    "SERVICIOS_MÉDICOS_AMBULATORIOS_62",
    "SERVICIOS_HOSPITALARIOS_63",
    "COMPRA_DE_VEHÍCULOS_71",
    "USO_DE_VEHÍCULOS_PERSONALES_72",
    "SERVICIOS_DE_TRANSPORTE_73",
    "SERVICIOS_POSTALES_81",
    "EQUIPOS_TELÉFONO_FAX_82",
    "SERVICIOS_TELÉFONO_FAX_83",
    "AUDIOVISUALES_TECNOLOGÍA_91",
    "BIENES_DURADEROS_DE_OCIO_92",
    "OCIO_JARDINERÍA_MASCOTAS_93",
    "SERVICIOS_RECREATIVOS_CULTURALES_94",
    "PRENSA_PAPELERÍA_95",
    "PAQUETES_TURÍSTICOS_96",
    "EDUCACIÓN_INFANTIL_PRIMARIA_101",
    "EDUCACIÓN_SECUNDARIA_POSTSECUNDARIA_102",
    "EDUCACIÓN_SUPERIOR_103",
    "EDUCACIÓN_NO_FORMAL_104",
    "RESTAURACIÓN_111",
    "ALOJAMIENTO_112",
    "CUIDADOS_PERSONALES_121",
    "EFECTOS_PERSONALES_123",
    "PROTECCIÓN_SOCIAL_124",
    "SEGUROS_125",
    "SERVICIOS_FINANCIEROS_126",
    "OTROS_SERVICIOS_127",
    "REMESAS_128"
]


In [326]:
columnas_prioritarias_camel = [
    "timestamp",

    "ccaa",
    "capitalProvincia",
    "tamanoMunicipio",
    "densidad",
    "superficie",
    "tipoCasa",
    "aguaCaliente",
    "calefaccion",
    "zonaResidencial",
    "regimenTenencia",
    "comidasTotales",

    "miembros:estudiantes",
    "miembros:noEstudiantes",
    "miembros:ocupados",
    "miembros:noOcupados",
    "miembros:activos",
    "miembros:noActivos",
    "miembros:ancianos:masculinos",
    "miembros:ancianos:femeninos",
    "miembros:adultos:masculinos",
    "miembros:adultos:femeninos",
    "miembros:ninos:femeninos",
    "miembros:ninos:masculinos",
    "miembros:conIngresos:masculinos",
    "miembros:conIngresos:femeninos",
    "miembros:sinIngresos:masculinos",
    "miembros:sinIngresos:femeninos",

    "trabajaSp",
    "edadSp",
    "espanolSp",
    "educacionSuperiorSp",

    "fuentePrincipalIngresos",
    "numeroViviendasAdicionales",
    "ingresosNetos",
    "gastoNoMonetario",
    "tasaAhorro",

    "temperaturaMedia",
    "tasaParo",
    "inflacion",
    "tipoInteres",
    "tasaCambioEurUsd",
    "ipc",

    "gastoMonetario:productosAlimenticios11",
    "gastoMonetario:bebidasNoAlcoholicas12",
    "gastoMonetario:bebidasAlcoholicas21",
    "gastoMonetario:tabaco22",
    "gastoMonetario:articulosDeVestir31",
    "gastoMonetario:calzado32",
    "gastoMonetario:alquileresRealesDeLaVivienda41",
    "gastoMonetario:mantenimientoDeLaVivienda43",
    "gastoMonetario:suministroDeAgua44",
    "gastoMonetario:electricidadGasOtrosCombustibles45",
    "gastoMonetario:mueblesRevestimientos51",
    "gastoMonetario:textilesParaElHogar52",
    "gastoMonetario:grandesElectrodomesticos53",
    "gastoMonetario:utensiliosDelHogar54",
    "gastoMonetario:herramientasCasaJardin55",
    "gastoMonetario:bienesServiciosParaElHogar56",
    "gastoMonetario:productosFarmaceuticos61",
    "gastoMonetario:serviciosMedicosAmbulatorios62",
    "gastoMonetario:serviciosHospitalarios63",
    "gastoMonetario:compraDeVehiculos71",
    "gastoMonetario:usoDeVehiculosPersonales72",
    "gastoMonetario:serviciosDeTransporte73",
    "gastoMonetario:serviciosPostales81",
    "gastoMonetario:equiposTelefonoFax82",
    "gastoMonetario:serviciosTelefonoFax83",
    "gastoMonetario:audiovisualesTecnologia91",
    "gastoMonetario:bienesDuraderosDeOcio92",
    "gastoMonetario:ocioJardineriaYMascotas93",
    "gastoMonetario:serviciosRecreativosYCulturales94",
    "gastoMonetario:prensaYPapeleria95",
    "gastoMonetario:paquetesTuristicos96",
    "gastoMonetario:educacionInfantilYPrimaria101",
    "gastoMonetario:educacionSecundariaYPostsecundaria102",
    "gastoMonetario:educacionSuperior103",
    "gastoMonetario:educacionNoFormal104",
    "gastoMonetario:restauracion111",
    "gastoMonetario:alojamiento112",
    "gastoMonetario:cuidadosPersonales121",
    "gastoMonetario:efectosPersonales123",
    "gastoMonetario:proteccionSocial124",
    "gastoMonetario:seguros125",
    "gastoMonetario:serviciosFinancieros126",
    "gastoMonetario:otrosServicios127",
    "gastoMonetario:remesas128",
]


In [327]:
print(len(columnas_prioritarias_camel))  
print(len(columnas_prioritarias))


87
87


In [328]:
def reordenar_columnas(df, columnas_prioritarias):
    # Asegurar que solo usamos las que realmente están en el df
    columnas_prioritarias = [col for col in columnas_prioritarias if col in df.columns]
    
    # Identificar columnas que no están en la lista prioritaria
    columnas_restantes = [col for col in df.columns if col not in columnas_prioritarias]
    
    # Orden final
    orden_final = columnas_prioritarias + columnas_restantes
    
    return df[orden_final]


In [329]:
df = reordenar_columnas(df_final_ajustado, columnas_prioritarias)


In [330]:
df.head(5)  # Mostrar las primeras filas del DataFrame reordenado

,Timestamp,CCAA,CAPROV,TAMAMU,DENSIDAD,SUPERF,TIPOCASA,AGUACALI,CALEF,ZONARES,REGTEN,COMITOT,NUMESTU,NUMNOESTU,NUMOCU,NUMNOOCU,NUMACTI,NUMNOACTI,ANC_MAS,ANC_FEM,NADUL_MAS,NADUL_FEM,NNINO_FEM,NNINO_MAS,PERCEP_MAS,PERCEP_FEM,NO_PERCEP_MAS,NO_PERCEP_FEM,EDADSP,NACION_ESP,EDUC_SUPERIOR,FUENPRINRED,NUMOVD,IMPEXAC,GASTNOMON,tasa_ahorro,T_med,Tasa_Paro,Inflacion,Tipo_Interes,EUR_USD,IPC,PRODUCTOS_ALIMENTICIOS_11,BEBIDAS_NO_ALCOHÓLICAS_12,BEBIDAS_ALCOHÓLICAS_21,TABACO_22,ARTÍCULOS_DE_VESTIR_31,CALZADO_32,ALQUILERES_REALES_DE_LA_VIVIENDA_41,MANTENIMIENTO_DE_LA_VIVIENDA_43,SUMINISTRO_DE_AGUA_44,ELECTRICIDAD_GAS_OTROS_COMBUSTIBLES_45,MUEBLES_REVESTIMIENTOS_51,TEXTILES_PARA_EL_HOGAR_52,GRANDES_ELECTRODOMÉSTICOS_53,UTENSILIOS_DEL_HOGAR_54,HERRAMIENTAS_CASA_JARDÍN_55,BIENES_SERVICIOS_PARA_EL_HOGAR_56,PRODUCTOS_FARMACÉUTICOS_61,SERVICIOS_MÉDICOS_AMBULATORIOS_62,SERVICIOS_HOSPITALARIOS_63,COMPRA_DE_VEHÍCULOS_71,USO_DE_VEHÍCULOS_PERSONALES_72,SERVICIOS_DE_TRANSPORTE_73,SERVICIOS_POSTALES_81,EQUIPOS_TELÉFONO_FAX_82,SERVICIOS_TELÉFONO_FAX_83,AUDIOVISUALES_TECNOLOGÍA_91,BIENES_DURADEROS_DE_OCIO_92,OCIO_JARDINERÍA_MASCOTAS_93,SERVICIOS_RECREATIVOS_CULTURALES_94,PRENSA_PAPELERÍA_95,PAQUETES_TURÍSTICOS_96,EDUCACIÓN_INFANTIL_PRIMARIA_101,EDUCACIÓN_SECUNDARIA_POSTSECUNDARIA_102,EDUCACIÓN_SUPERIOR_103,EDUCACIÓN_NO_FORMAL_104,RESTAURACIÓN_111,ALOJAMIENTO_112,CUIDADOS_PERSONALES_121,EFECTOS_PERSONALES_123,PROTECCIÓN_SOCIAL_124,SEGUROS_125,SERVICIOS_FINANCIEROS_126,OTROS_SERVICIOS_127,REMESAS_128
0,2006-01-01T00:00:00Z,9,0,4,3,70.0,2,1,1,5,5,58.0,0,2,2.0,0,2.0,0,0,0,0,0,1,1,1,1,0,0,55,1,1,autonomYRenta,0.0,42599.711960,4632.676246,0.325187,17.62,6.5,2.8,3.5,1.26,78.512833,5366.587817,NaN,2501.325816,NaN,268.610136,NaN,NaN,849.123143,172.831501,350.980894,NaN,NaN,2300.023791,NaN,464.389191,1183.669487,181.048776,1471.132682,NaN,NaN,225.161824,2235.473630,105.643608,26.186377,1520.690913,757.453145,NaN,991.716203,529.568730,1903.565017,NaN,NaN,NaN,141.680815,NaN,84.442756,NaN,3246.912707,NaN,NaN,1868.602704,NaN,NaN,NaN
1,2006-01-01T00:00:00Z,9,1,5,3,65.0,2,1,1,5,4,36.0,0,2,1.0,1,1.0,1,0,0,0,1,1,0,0,1,0,1,49,1,1,asalariado,0.0,14095.243340,0.000000,-0.260235,17.62,6.5,2.8,3.5,1.26,78.512833,3722.156904,981.893660,68.588082,59.599995,2056.440279,26.759182,2011.181620,NaN,89.102983,377.626929,6.279639,96.019072,NaN,12.071333,3.380256,274.189680,1232.852919,3.564119,NaN,NaN,157.422342,198.239994,NaN,NaN,803.631120,19.574370,NaN,2.772093,582.167777,864.334295,173.970039,NaN,1018.319808,NaN,NaN,1447.251669,0.650593,979.552938,13.323017,NaN,476.276660,NaN,4.129853,NaN
2,2006-01-01T00:00:00Z,9,0,1,1,120.0,3,1,1,1,5,72.0,0,4,1.0,3,1.0,3,0,0,1,1,1,1,1,0,1,2,38,1,0,asalariado,0.0,9181.850267,3406.873777,-0.412638,17.62,6.5,2.8,3.5,1.26,78.512833,1627.600349,124.998756,31.893211,964.985308,1311.320118,1130.391560,65.448263,23.230421,354.283361,431.979748,59.366630,21.568579,34.474368,270.724564,8.295063,59.918220,36.136210,66.742377,9.589178,507.978936,412.751890,33.555052,NaN,1.477978,334.369552,1441.014460,2.765021,105.643608,639.575556,68.029420,53.468862,242.239896,2.948884,5.162316,1.661841,309.922805,1199.764597,452.353216,31.893211,22.749547,423.168453,NaN,43.328093,1.845705
3,2006-01-01T00:00:00Z,9,0,1,2,70.0,2,1,1,3,6,60.0,0,2,0.0,2,0.0,2,1,1,0,0,0,0,1,1,0,0,74,1,0,pension,0.0,10879.049947,3126.581250,-0.513042,17.62,6.5,2.8,3.5,1.26,78.512833,3232.896667,875.747963,345.507425,NaN,1056.549231,NaN,NaN,107.432740,202.956795,1417.444599,NaN,NaN,NaN,NaN,NaN,115.048923,35.952347,416.344296,NaN,NaN,5475.024733,NaN,NaN,NaN,833.112893,NaN,NaN,537.248559,NaN,268.440416,NaN,NaN,NaN,NaN,NaN,176.438050,NaN,947.086923,NaN,NaN,417.228255,NaN,NaN,NaN
4,2006-01-01T00:00:00Z,9,0,4,3,56.0,2,1,1,5,6,23.0,0,1,0.0,1,0.0,1,0,0,0,0,0,1,1,0,0,0,48,1,1,pension,0.0,14298.907301,3910.178342,0.608033,17.62,6.5,2.8,3.5,1.26,78.512833,1232.867062,NaN,NaN,290.376722,NaN,NaN,NaN,NaN,843.034439,364.049331,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,780.626993,NaN,NaN,14.192832,NaN

In [331]:
# Creamos el diccionario de renombrado
rename_dict = dict(zip(columnas_prioritarias, columnas_prioritarias_camel))

# Aplicarlo al DataFrame (por ejemplo df_final_ajustado)
df_final = df.rename(columns=rename_dict)

In [332]:
rename_dict

{'Timestamp': 'timestamp',
 'CCAA': 'ccaa',
 'CAPROV': 'capitalProvincia',
 'TAMAMU': 'tamanoMunicipio',
 'DENSIDAD': 'densidad',
 'SUPERF': 'superficie',
 'TIPOCASA': 'tipoCasa',
 'AGUACALI': 'aguaCaliente',
 'CALEF': 'calefaccion',
 'ZONARES': 'zonaResidencial',
 'REGTEN': 'regimenTenencia',
 'COMITOT': 'comidasTotales',
 'NUMESTU': 'miembros:estudiantes',
 'NUMNOESTU': 'miembros:noEstudiantes',
 'NUMOCU': 'miembros:ocupados',
 'NUMNOOCU': 'miembros:noOcupados',
 'NUMACTI': 'miembros:activos',
 'NUMNOACTI': 'miembros:noActivos',
 'ANC_MAS': 'miembros:ancianos:masculinos',
 'ANC_FEM': 'miembros:ancianos:femeninos',
 'NADUL_MAS': 'miembros:adultos:masculinos',
 'NADUL_FEM': 'miembros:adultos:femeninos',
 'NNINO_FEM': 'miembros:ninos:femeninos',
 'NNINO_MAS': 'miembros:ninos:masculinos',
 'PERCEP_MAS': 'miembros:conIngresos:masculinos',
 'PERCEP_FEM': 'miembros:conIngresos:femeninos',
 'NO_PERCEP_MAS': 'miembros:sinIngresos:masculinos',
 'NO_PERCEP_FEM': 'miembros:sinIngresos:femeninos'

Ajustamos nuestro codigos de comunidad a dos digitos

In [333]:
df_final['ccaa'] = df_final['ccaa'].astype(str).str.zfill(2)

In [334]:
df_final.head()  

,timestamp,ccaa,capitalProvincia,tamanoMunicipio,densidad,superficie,tipoCasa,aguaCaliente,calefaccion,zonaResidencial,regimenTenencia,comidasTotales,miembros:estudiantes,miembros:noEstudiantes,miembros:ocupados,miembros:noOcupados,miembros:activos,miembros:noActivos,miembros:ancianos:masculinos,miembros:ancianos:femeninos,miembros:adultos:masculinos,miembros:adultos:femeninos,miembros:ninos:femeninos,miembros:ninos:masculinos,miembros:conIngresos:masculinos,miembros:conIngresos:femeninos,miembros:sinIngresos:masculinos,miembros:sinIngresos:femeninos,edadSp,espanolSp,educacionSuperiorSp,fuentePrincipalIngresos,numeroViviendasAdicionales,ingresosNetos,gastoNoMonetario,tasaAhorro,temperaturaMedia,tasaParo,inflacion,tipoInteres,tasaCambioEurUsd,ipc,gastoMonetario:productosAlimenticios11,gastoMonetario:bebidasNoAlcoholicas12,gastoMonetario:bebidasAlcoholicas21,gastoMonetario:tabaco22,gastoMonetario:articulosDeVestir31,gastoMonetario:calzado32,gastoMonetario:alquileresRealesDeLaVivienda41,gastoMonetario:mantenimientoDeLaVivienda43,gastoMonetario:suministroDeAgua44,gastoMonetario:electricidadGasOtrosCombustibles45,gastoMonetario:mueblesRevestimientos51,gastoMonetario:textilesParaElHogar52,gastoMonetario:grandesElectrodomesticos53,gastoMonetario:utensiliosDelHogar54,gastoMonetario:herramientasCasaJardin55,gastoMonetario:bienesServiciosParaElHogar56,gastoMonetario:productosFarmaceuticos61,gastoMonetario:serviciosMedicosAmbulatorios62,gastoMonetario:serviciosHospitalarios63,gastoMonetario:compraDeVehiculos71,gastoMonetario:usoDeVehiculosPersonales72,gastoMonetario:serviciosDeTransporte73,gastoMonetario:serviciosPostales81,gastoMonetario:equiposTelefonoFax82,gastoMonetario:serviciosTelefonoFax83,gastoMonetario:audiovisualesTecnologia91,gastoMonetario:bienesDuraderosDeOcio92,gastoMonetario:ocioJardineriaYMascotas93,gastoMonetario:serviciosRecreativosYCulturales94,gastoMonetario:prensaYPapeleria95,gastoMonetario:paquetesTuristicos96,gastoMonetario:educacionInfantilYPrimaria101,gastoMonetario:educacionSecundariaYPostsecundaria102,gastoMonetario:educacionSuperior103,gastoMonetario:educacionNoFormal104,gastoMonetario:restauracion111,gastoMonetario:alojamiento112,gastoMonetario:cuidadosPersonales121,gastoMonetario:efectosPersonales123,gastoMonetario:proteccionSocial124,gastoMonetario:seguros125,gastoMonetario:serviciosFinancieros126,gastoMonetario:otrosServicios127,gastoMonetario:remesas128
0,2006-01-01T00:00:00Z,09,0,4,3,70.0,2,1,1,5,5,58.0,0,2,2.0,0,2.0,0,0,0,0,0,1,1,1,1,0,0,55,1,1,autonomYRenta,0.0,42599.711960,4632.676246,0.325187,17.62,6.5,2.8,3.5,1.26,78.512833,5366.587817,NaN,2501.325816,NaN,268.610136,NaN,NaN,849.123143,172.831501,350.980894,NaN,NaN,2300.023791,NaN,464.389191,1183.669487,181.048776,1471.132682,NaN,NaN,225.161824,2235.473630,105.643608,26.186377,1520.690913,757.453145,NaN,991.716203,529.568730,1903.565017,NaN,NaN,NaN,141.680815,NaN,84.442756,NaN,3246.912707,NaN,NaN,1868.602704,NaN,NaN,NaN
1,2006-01-01T00:00:00Z,09,1,5,3,65.0,2,1,1,5,4,36.0,0,2,1.0,1,1.0,1,0,0,0,1,1,0,0,1,0,1,49,1,1,asalariado,0.0,14095.243340,0.000000,-0.260235,17.62,6.5,2.8,3.5,1.26,78.512833,3722.156904,981.893660,68.588082,59.599995,2056.440279,26.759182,2011.181620,NaN,89.102983,377.626929,6.279639,96.019072,NaN,12.071333,3.380256,274.189680,1232.852919,3.564119,NaN,NaN,157.422342,198.239994,NaN,NaN,803.631120,19.574370,NaN,2.772093,582.167777,864.334295,173.970039,NaN,1018.319808,NaN,NaN,1447.251669,0.650593,979.552938,13.323017,NaN,476.276660,NaN,4.129853,NaN
2,2006-01-01T00:00:00Z,09,0,1,1,120.0,3,1,1,1,5,72.0,0,4,1.0,3,1.0,3,0,0,1,1,1,1,1,0,1,2,38,1,0,asalariado,0.0,9181.850267,3406.873777,-0.412638,17.62,6.5,2.8,3.5,1.26,78.512833,1627.600349,124.998756,31.893211,964.985308,1311.320118,1130.391560,65.448263,23.230421,354.283361,431.979748,59.366630,21.568579,34.474368,270.724564,8.295063,59.918220,36.136210,66.742377,9.589178,507.978936,412.751890,33.555052,NaN,1.477978,334.369552,1441.014460,2.765021,105.643608,639.575556,68.029420,53.468862,

In [335]:
df_final.tail()

,timestamp,ccaa,capitalProvincia,tamanoMunicipio,densidad,superficie,tipoCasa,aguaCaliente,calefaccion,zonaResidencial,regimenTenencia,comidasTotales,miembros:estudiantes,miembros:noEstudiantes,miembros:ocupados,miembros:noOcupados,miembros:activos,miembros:noActivos,miembros:ancianos:masculinos,miembros:ancianos:femeninos,miembros:adultos:masculinos,miembros:adultos:femeninos,miembros:ninos:femeninos,miembros:ninos:masculinos,miembros:conIngresos:masculinos,miembros:conIngresos:femeninos,miembros:sinIngresos:masculinos,miembros:sinIngresos:femeninos,edadSp,espanolSp,educacionSuperiorSp,fuentePrincipalIngresos,numeroViviendasAdicionales,ingresosNetos,gastoNoMonetario,tasaAhorro,temperaturaMedia,tasaParo,inflacion,tipoInteres,tasaCambioEurUsd,ipc,gastoMonetario:productosAlimenticios11,gastoMonetario:bebidasNoAlcoholicas12,gastoMonetario:bebidasAlcoholicas21,gastoMonetario:tabaco22,gastoMonetario:articulosDeVestir31,gastoMonetario:calzado32,gastoMonetario:alquileresRealesDeLaVivienda41,gastoMonetario:mantenimientoDeLaVivienda43,gastoMonetario:suministroDeAgua44,gastoMonetario:electricidadGasOtrosCombustibles45,gastoMonetario:mueblesRevestimientos51,gastoMonetario:textilesParaElHogar52,gastoMonetario:grandesElectrodomesticos53,gastoMonetario:utensiliosDelHogar54,gastoMonetario:herramientasCasaJardin55,gastoMonetario:bienesServiciosParaElHogar56,gastoMonetario:productosFarmaceuticos61,gastoMonetario:serviciosMedicosAmbulatorios62,gastoMonetario:serviciosHospitalarios63,gastoMonetario:compraDeVehiculos71,gastoMonetario:usoDeVehiculosPersonales72,gastoMonetario:serviciosDeTransporte73,gastoMonetario:serviciosPostales81,gastoMonetario:equiposTelefonoFax82,gastoMonetario:serviciosTelefonoFax83,gastoMonetario:audiovisualesTecnologia91,gastoMonetario:bienesDuraderosDeOcio92,gastoMonetario:ocioJardineriaYMascotas93,gastoMonetario:serviciosRecreativosYCulturales94,gastoMonetario:prensaYPapeleria95,gastoMonetario:paquetesTuristicos96,gastoMonetario:educacionInfantilYPrimaria101,gastoMonetario:educacionSecundariaYPostsecundaria102,gastoMonetario:educacionSuperior103,gastoMonetario:educacionNoFormal104,gastoMonetario:restauracion111,gastoMonetario:alojamiento112,gastoMonetario:cuidadosPersonales121,gastoMonetario:efectosPersonales123,gastoMonetario:proteccionSocial124,gastoMonetario:seguros125,gastoMonetario:serviciosFinancieros126,gastoMonetario:otrosServicios127,gastoMonetario:remesas128
365313,2023-01-01T00:00:00Z,18,1,4,3,82.0,2,1,0,4,3,112.0,0,8,0.0,8,4.0,4,0,0,2,2,0,4,1,1,5,1,43,0,0,pension,0.0,12960.0,0.0,-0.766272,19.6,30.0,3.3,4.5,1.08,112.407167,5646.01,1503.59,NaN,NaN,8760.02,1460.00,1176.0,NaN,312.0,540.0,NaN,NaN,NaN,NaN,NaN,216.69,NaN,262.75,NaN,NaN,1298.41,NaN,NaN,NaN,360.0,NaN,NaN,NaN,NaN,660.36,NaN,NaN,NaN,NaN,NaN,375.04,NaN,NaN,NaN,NaN,NaN,320.0,NaN,NaN
365314,2023-01-01T00:00:00Z,18,1,4,3,200.0,3,1,0,6,3,34.0,0,3,0.0,3,1.0,2,1,1,1,0,0,0,2,0,0,1,85,1,1,pension,0.0,44160.0,0.0,0.094063,19.6,30.0,3.3,4.5,1.08,112.407167,10612.64,4202.06,1360.66,1132.79,216.69,193.80,2880.0,NaN,540.0,1260.0,NaN,NaN,129.2,NaN,NaN,351.41,3278.62,516.81,NaN,NaN,NaN,NaN,NaN,NaN,672.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9920.55,NaN,2738.96,NaN,NaN,NaN,NaN,NaN,NaN
365315,2023-01-01T00:00:00Z,18,1,4,3,70.0,2,1,0,5,3,42.0,0,3,0.0,3,2.0,1,0,0,1,1,1,0,1,0,0,2,30,1,0,pension,0.0,5472.0,0.0,-0.044229,19.6,30.0,3.3,4.5,1.08,112.407167,2778.56,NaN,NaN,NaN,NaN,NaN,NaN,NaN,456.0,726.6,NaN,NaN,NaN,NaN,NaN,NaN,1401.36,NaN,NaN,NaN,NaN,255.50,NaN,NaN,96.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
365316,2023-01-01T00:00:00Z,18,1,4,3,128.0,2,1,0,6,6,48.0,1,3,2.0,2,2.0,2,0,0,1,1,0,2,1,1,2,0,64,1,1,asalariado,0.0,96000.0,9531.9,0.637511,19.6,30.0,3.3,4.5,1.08,112.407167,8035.61,835.01,232.56,NaN,9879.36,1216.67,NaN,NaN,2400.0,2160.0,NaN,NaN,1967.6,NaN,NaN,1142.88,1313.77,NaN,NaN,NaN,NaN,851.67,NaN,NaN,1920.0,NaN,NaN,NaN,NaN,218.96,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1824.81,NaN,NaN,NaN,800.0,NaN,NaN
365317,2023-01-01T00:00:00Z,18,1,4,3,4

### Datamart para cada comunidad

In [336]:
import os

# Directorio donde van a guardarse los ficheros
output_dir = '../../datamarts/picotaData'
os.makedirs(output_dir, exist_ok=True)

for ccaa, group in df_final.groupby('ccaa'):
    df_sub = group.drop(columns=['ccaa'])
    filename = f'hogar{ccaa}.tsv'
    filepath = os.path.join(output_dir, filename)
    df_sub.to_csv(filepath, sep='\t', index=False)

print("Ficheros guardados en", output_dir)


Ficheros guardados en ../../datamarts/picotaData


In [337]:
# Cargar de la ruta de salida un fichero de ejemplo
df_example = pd.read_csv(os.path.join(output_dir, 'hogar01.tsv'), sep='\t')
df_example.head()

,timestamp,capitalProvincia,tamanoMunicipio,densidad,superficie,tipoCasa,aguaCaliente,calefaccion,zonaResidencial,regimenTenencia,comidasTotales,miembros:estudiantes,miembros:noEstudiantes,miembros:ocupados,miembros:noOcupados,miembros:activos,miembros:noActivos,miembros:ancianos:masculinos,miembros:ancianos:femeninos,miembros:adultos:masculinos,miembros:adultos:femeninos,miembros:ninos:femeninos,miembros:ninos:masculinos,miembros:conIngresos:masculinos,miembros:conIngresos:femeninos,miembros:sinIngresos:masculinos,miembros:sinIngresos:femeninos,edadSp,espanolSp,educacionSuperiorSp,fuentePrincipalIngresos,numeroViviendasAdicionales,ingresosNetos,gastoNoMonetario,tasaAhorro,temperaturaMedia,tasaParo,inflacion,tipoInteres,tasaCambioEurUsd,ipc,gastoMonetario:productosAlimenticios11,gastoMonetario:bebidasNoAlcoholicas12,gastoMonetario:bebidasAlcoholicas21,gastoMonetario:tabaco22,gastoMonetario:articulosDeVestir31,gastoMonetario:calzado32,gastoMonetario:alquileresRealesDeLaVivienda41,gastoMonetario:mantenimientoDeLaVivienda43,gastoMonetario:suministroDeAgua44,gastoMonetario:electricidadGasOtrosCombustibles45,gastoMonetario:mueblesRevestimientos51,gastoMonetario:textilesParaElHogar52,gastoMonetario:grandesElectrodomesticos53,gastoMonetario:utensiliosDelHogar54,gastoMonetario:herramientasCasaJardin55,gastoMonetario:bienesServiciosParaElHogar56,gastoMonetario:productosFarmaceuticos61,gastoMonetario:serviciosMedicosAmbulatorios62,gastoMonetario:serviciosHospitalarios63,gastoMonetario:compraDeVehiculos71,gastoMonetario:usoDeVehiculosPersonales72,gastoMonetario:serviciosDeTransporte73,gastoMonetario:serviciosPostales81,gastoMonetario:equiposTelefonoFax82,gastoMonetario:serviciosTelefonoFax83,gastoMonetario:audiovisualesTecnologia91,gastoMonetario:bienesDuraderosDeOcio92,gastoMonetario:ocioJardineriaYMascotas93,gastoMonetario:serviciosRecreativosYCulturales94,gastoMonetario:prensaYPapeleria95,gastoMonetario:paquetesTuristicos96,gastoMonetario:educacionInfantilYPrimaria101,gastoMonetario:educacionSecundariaYPostsecundaria102,gastoMonetario:educacionSuperior103,gastoMonetario:educacionNoFormal104,gastoMonetario:restauracion111,gastoMonetario:alojamiento112,gastoMonetario:cuidadosPersonales121,gastoMonetario:efectosPersonales123,gastoMonetario:proteccionSocial124,gastoMonetario:seguros125,gastoMonetario:serviciosFinancieros126,gastoMonetario:otrosServicios127,gastoMonetario:remesas128
0,2006-01-01T00:00:00Z,0,1,1,80.0,1,1,0,3,4,84.0,0,3,0.0,3,1.0,2,1,1,0,0,1,0,1,1,0,1,67,1,0,pension,0.0,5478.930437,0.000000,0.148916,19.62,12.6,2.9,3.5,1.26,81.54775,984.610902,13.234969,29.135715,117.836016,133.888473,82.595743,260.075812,3.041442,115.589250,641.151895,4.370719,14.101888,1.054752,16.471468,21.463479,76.809056,73.854306,11.826225,NaN,NaN,322.111117,19.881351,NaN,NaN,307.857518,3.749426,NaN,41.359278,164.418483,64.527699,6.299614,NaN,NaN,NaN,NaN,509.170645,NaN,453.788946,6.234595,NaN,156.045487,NaN,NaN,6.465774
1,2006-01-01T00:00:00Z,0,3,1,90.0,2,1,0,5,6,68.0,2,3,3.0,2,3.0,2,0,0,0,2,1,2,2,1,0,2,47,1,0,asalariado,0.0,25097.315845,2766.339719,0.636083,19.62,12.6,2.9,3.5,1.26,81.54775,1091.653772,32.386663,208.876999,56.385881,705.903548,736.513026,NaN,NaN,183.411242,821.882911,0.065019,1.936120,NaN,17.143331,0.086692,192.911233,179.849648,235.874313,NaN,NaN,2210.391549,34.886280,NaN,NaN,909.167244,17.547893,0.527376,13.487821,204.426812,106.710550,0.628517,NaN,NaN,NaN,1.502660,514.162656,NaN,149.290740,10.923184,NaN,484.196143,NaN,10.612538,NaN
2,2006-01-01T00:00:00Z,0,2,2,90.0,2,1,0,5,6,56.0,0,2,0.0,2,0.0,2,1,1,0,0,0,0,1,0,0,1,77,1,0,pension,0.0,3918.475565,1995.561704,-0.217122,19.62,12.6,2.9,3.5,1.26,81.54775,2188.985865,135.795695,77.033011,NaN,329.696662,NaN,NaN,NaN,120.588485,544.829929,NaN,NaN,248.495214,NaN,NaN,54.810977,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,173.773988,NaN,NaN,NaN,320.738495,NaN,NaN,NaN,NaN,NaN,NaN,149.550816,NaN,289.204303,NaN,NaN,135.759574,NaN,NaN,NaN
3,2006-01-01T00:00:00Z,1,5,3,90.0,1,1,1,4,2,10.0,0,1,0.0,1,0.0,1,0,0,0,0,1